In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn import metrics
# from sklearn.metrics import r2_score
import math
import statsmodels.api as sm
from numpy import log as ln
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings("ignore")

path = '/Users/mirac/Desktop/Chlorine_Project/August 18 Chlorine Review Paper/Chlorine_Review_Revisions/1-Revisions_Inactivation_Rate_Data_R1.xlsx'

df = pd.read_excel(path)

# Functions

### There are only 3 methods:
Method 1 - chlorine decay (The code will take care of the case of k prime given or not)

Method 2 - Constant Chlorine

Method 3 - Given CT

Method 4 - k values are given

In [2]:
# REQUIRE: df consists of every data point
# MODIFY: Break down into several dataframe and fill up the Sample ID columns
# EFFECT: return a list with subset dataframes of the data
def get_df_list(df):
    df_list = []
    ID_ind_list = df['Sample_ID'][df['Sample_ID'].notnull()].index.tolist()
    for i in range(len(ID_ind_list)):
        if (i != len(ID_ind_list)-1):
            start_ind = ID_ind_list[i]
            end_ind = ID_ind_list[i+1]
        else:
            start_ind = ID_ind_list[i]
            end_ind = df.shape[0]

        df_copy = df.copy()
        df_temp = df_copy.iloc[start_ind:end_ind]
        ID_label = df_copy.iloc[start_ind:end_ind]['Sample_ID'].unique()[0]
        df_temp['Sample_ID'] = ID_label
        df_temp = df_temp.reset_index(drop = True)
        df_list.append(df_temp)
    return df_list

# Get input function variables for each method
def get_info(df, k_is_given):

    if k_is_given:
        method = 0
        have_k_prime_already = False
        C_init = 0
        const_C = 0 
        num_pt = 0

        return method, have_k_prime_already, C_init, const_C, num_pt

    else: 
        # Get method
        method = int(df['Method'][0])
        
        # Get k_prime
        given_k_prime = np.isnan(df['k_prime'][0])
        if given_k_prime == False: # is not NaN
            have_k_prime_already = df['k_prime'][0] 
        else:
            have_k_prime_already = False
        
        # Get C0
        given_C0 = np.isnan(df['C0'][0])
        if given_C0 == False: # is not NaN
            C_init = float(df['C0'][0])
        else:
            C_init = False
        
        # Get constant chlorine
        given_const_C = np.isnan(df['Constant_Chlorine'][0])
        if given_const_C == False: # is not NaN
            const_C = float(df['Constant_Chlorine'][0])
        else:
            const_C = False
        
        # Get num_point used
        num_pt = int(df['Data_Points_Used'][0])
        
        return method, have_k_prime_already, C_init, const_C, num_pt 


# Check the coef calculated from linear regression from numpy and OLS
# EFFECT: return True if they are the same (with difference smaller than tolerance of 1e-5) and return False if diff > tolerance
def same_coef(coef, coef_ols):
    diff = coef - coef_ols
    tolerance = 1e-5
    if (diff < tolerance):
        return True
    else:
        return False

# Display summary df after extracting the coefs from np, ols, and standard errors    
def display_summary_df(Sample_ID_list, method_list, coef_list, coef_ols_list, error_list, totalPointUsed_list):
    
    def modify_diff(x):
        x = float(np.abs(x))
        tol = 10e-6
        if (x < tol):
            return 0
        else:
            return x
    
    summ = pd.DataFrame()
    summ['Sample_ID'] = Sample_ID_list
    summ['Method'] = method_list
    summ['Coef_np'] = coef_list
    summ['Coef_OLS'] = coef_ols_list
    summ['Standard_Error'] = error_list
    summ['Data_Points_Used'] = totalPointUsed_list
    
    summ['Diff'] = summ['Coef_np'] - summ['Coef_OLS']
    summ['Diff'] = summ['Diff'].apply(modify_diff)
    
    return summ   

def results(path, sheet_name):
    df_whole = pd.read_excel(path, sheet_name) # Get each tab or sheet

    if (df_whole.shape[0] == 0):
        return pd.DataFrame()

    defaultCols = [ 'Time (min)', 'C (mg/L)', 'ln(C/C0)', 'Log 10 reduction', 'N/N0',
                    'Titer (PFU/mL)', 'CT_values', 'ln 10 reduction', 'Method', 'k_prime',
                    'C0', 'Constant_Chlorine', 'Data_Points_Used', 'Sample_ID', 'Given_k']

    extraCols = list(set(df_whole.columns) - set(defaultCols))
    # print(extraCols) # testing
    # display(df_whole)
    
    # if len(extraCols) > 0:
    #     if (df_whole[extraCols].dropna().shape[0]) == 0:
    #         extraCols = []

    df_list = get_df_list(df_whole)

    if len(extraCols) > 0:
        extra_df_cols = extraCols + ['Sample_ID']
        extraFilter = df_whole[extra_df_cols]
        extraFilter.dropna(subset = ['Sample_ID'], inplace=True)

    method_list = []
    Sample_ID_list = []

    for i in range(len(df_list)):
        each_df = df_list[i] # get each subset of datapoints

        k_is_given = (each_df['Given_k'].isnull()[0] == False)

        if (k_is_given):
            method_list.append(0) # Approach 0 is when k values are given
        else:
            method_temp = int(each_df['Method'][0])
            method_list.append(method_temp)

        Sample_ID_temp = str(each_df['Sample_ID'][0])
        Sample_ID_list.append(Sample_ID_temp)

    coef_list = []
    coef_ols_list = []
    error_list = []
    totalPointUsed_list = []

    for method, temp in zip(method_list, df_list): # for each paper 
        
        k_is_given = (temp['Given_k'].isnull()[0] == False)
        method, have_k_prime_already, C_init, const_C, num_pt = get_info(temp, k_is_given)

        if (method == 1):
            coef, coef_ols, error = chlorine_decay(temp, have_k_prime_already, C_init, num_pt)
        elif (method == 2):
            coef, coef_ols, error = constant_chlorine(temp, const_C, num_pt)
        elif (method == 3):
            coef, coef_ols, error = Given_CT(temp, num_pt)
        elif (method == 0):
            k_value = Given_K(temp)
            coef = k_value
            coef_ols = k_value
            error = 0

        coef_list.append(coef)
        coef_ols_list.append(coef_ols)
        error_list.append(error)
        totalPointUsed_list.append(num_pt)

    summ = display_summary_df(Sample_ID_list, method_list, coef_list, coef_ols_list, error_list, totalPointUsed_list)

    if (len(extraCols) > 0):
        if (summ.shape[0] != extraFilter.shape[0]):
            print("extraCols: ", extraCols)
            print(f"summ shape: {summ.shape}")
            print(f"extraFilter shape: {extraFilter.shape}")
            display(summ)
            display(extraFilter)
            raise ValueError("Dataframe summ and extraFilter have different shapes!")
        summ = summ.merge(extraFilter, on = 'Sample_ID', how = 'left') 
        
    return summ
#########################################################################################################

# For Approach 3

# Plot the graph: time vs ln(C/C0)
def plot_lnC_vs_time(df):
    fig, ax = plt.subplots(figsize = (10,6))
    plt.scatter(df['Time (min)'], df['ln(C/C0)'], c = 'red')
    plt.title("Choose points to get the k' value", fontweight="bold", fontsize = 14)
    plt.xlabel('Time (min)', fontsize = 14); plt.ylabel('ln(C/C0)', fontsize = 14); plt.grid(True)
    
    for i in range(len(df['ln(C/C0)'])):
        ax.annotate(str(i+1), (df['Time (min)'][i], df['ln(C/C0)'][i]), 
                    xytext = (df['Time (min)'][i], df['ln(C/C0)'][i]),
                    fontsize = 14, color = "black")
    plt.show()
    

# For Approach 3 to calculate the coef of k   
def get_k_prime_3(df, k_primes_to_use):
    # display(df) # testing
    x = np.array(df['Time (min)'][0:k_primes_to_use])
    y = np.array(df['ln(C/C0)'][0:k_primes_to_use])
    A = np.vstack([x, np.ones(len(x))]).T # Add one to x and drop the ones later
    A = A.astype('float64') 
    k_prime_3 = float(np.linalg.lstsq(A[:, :-1], y)[0]) # use [0] to just grab the coefs  
    
    df_xy = pd.DataFrame({"x_val": x, "y_val": y})
    smOLS_reault_k_prime = sm.OLS(endog= df_xy[['y_val']], exog= df_xy[['x_val']].assign(intercept=0)).fit() 
    # intercept=0 makes sure the line passes through the origin

    k_prime_OLS = smOLS_reault_k_prime.params[0]
    std_error = smOLS_reault_k_prime.bse[0]
    
    print("================================================================")
    print("Coef for k' value (OLS): ", smOLS_reault_k_prime.params[0])
    print("Standard Error for k' value (OLS): ", smOLS_reault_k_prime.bse[0])
    print("================================================================")

    return k_prime_3


# Plot the Ln vs CT graph
def plot_ln_vs_CT(df, CT_assign): #### Note: CT_assign is CT Approachg 3; just put "CT_values_3" during function call
    fig, ax = plt.subplots(figsize = (10,6)) #### Note: might need to move this line before calling this function
    plt.scatter(df[CT_assign], df['ln 10 reduction'], c = 'red')
    for i in range(len(df[CT_assign])):
        ax.annotate(str(i+1), (df[CT_assign][i], df['ln 10 reduction'][i]), 
                    xytext = (df[CT_assign][i], df['ln 10 reduction'][i]),
                    fontsize = 14)
    
    plt.title("Choose CT Values", fontweight="bold", fontsize = 14)
    plt.xlabel(CT_assign, fontsize = 14); plt.ylabel("ln", fontsize = 14);plt.grid(True)
    plt.show()
    
    
def get_coef(df, CT_approach_num, CT, CT_to_use):
    if (df[CT].isnull().sum() >= len(df)-1):
        coef = 0
    else:      
        if (df["ln 10 reduction"].isnull().sum() != 0):
            data = df.dropna(subset = ['ln 10 reduction'])
        else: 
            data = df.copy()

        x = np.array(data[CT][0:CT_to_use])
        y = np.array(data['ln 10 reduction'][0:CT_to_use])
        A = np.vstack([x, np.ones(len(x))]).T # Add one to x and drop the ones later
        
        coef = float(np.linalg.lstsq(A[:, :-1], y)[0]) # use [0] to just grab the coefs
    
    return coef


def visualize_linear_regression(X, Y, CT_approach_num, CT, coef, point_style, line_style):   
    plt.plot(X, Y, point_style, label = CT)           # plot the data points 
    plt.plot(X, X * coef, line_style) # Plot the linear regression
        
    
def get_R2(y_actual, y_pred):
    mean_y_actual = np.mean(y_actual)
    numerator = 0
    denominator = 0
    for y_a, y_p in zip(y_actual, y_pred):
        numerator += (y_a - y_p)**2
        denominator += (y_a - mean_y_actual)**2
    ratio = numerator/denominator
    r2 = 1- ratio

    return r2


def summary(coef, C_0, CT_to_use, CT_approach_num, CT, ds):
    ds = ds.iloc[0:CT_to_use,:]
    
    y_predicted_name = "y_predicted_" + str(CT_approach_num)
    ds[y_predicted_name] = ds[CT] * coef
    
    residual_name = "residual_" + str(CT_approach_num)
    ds[residual_name] = ds['ln 10 reduction'] - ds[y_predicted_name] 
    
    # Calculate R2
    R2 = get_R2(np.array(ds['ln 10 reduction']), np.array(ds[y_predicted_name]))
    R2_name = "R Square_" + str(CT_approach_num)
    ds[R2_name] = R2
    
    return ds, R2

##### UPDATED ON 6/25/2022 ###########

def convert_to_ln(df): 
    
    # Case: have only Titer (PFU/mL)
    if (df["Titer (PFU/mL)"].isnull().sum() != len(df)):
        inital_titer = df["Titer (PFU/mL)"][0]
        df["N/N0"] = df["Titer (PFU/mL)"]/inital_titer

    # Case: have only Log 10 reduction
    if (df["N/N0"].isnull().sum() == len(df)) & (df["ln 10 reduction"].isnull().sum() == len(df)) & (df["Log 10 reduction"].isnull().sum() != len(df)):
        df['ln 10 reduction'] = df['Log 10 reduction'] * math.log(10)
        
    # Case: have only N/N0
    elif (df["N/N0"].isnull().sum() != len(df)) & (df["ln 10 reduction"].isnull().sum() == len(df)) & (df["Log 10 reduction"].isnull().sum() == len(df)): 
        ln_list = [math.log(x) for x in df["N/N0"]]
        df["ln 10 reduction"] = ln_list 

    # Case: have only ln 10 reduction
    elif (df["N/N0"].isnull().sum() == len(df)) & (df["ln 10 reduction"].isnull().sum() != len(df)) & (df["Log 10 reduction"].isnull().sum() == len(df)): 
        df['ln 10 reduction'] = df['ln 10 reduction'] 
    
    # Update constrain based on paper 23 (9/15/2022)
    # Case: have only C/C0
    elif (df["ln(C/C0)"].isnull().sum() != len(df)) & (df["N/N0"].isnull().sum() == len(df)) & (df["ln 10 reduction"].isnull().sum() == len(df)) & (df["Log 10 reduction"].isnull().sum() == len(df)): 
        df['ln 10 reduction'] = df['ln(C/C0)']   # Heree  

        
def empty_df(df):
    for col in df.columns:
        df[col] = np.nan
    return df

# Sort rows according to Time (min) for every case except for Given CT        
def sort_row_according_to_Time(df):
    df_sorted = df.sort_values(by = "Time (min)", ascending = True)
    return df_sorted

# Sort rows according to CT_values for the case of Given CT     
def sort_row_according_to_CT(df):
    df_sorted = df.sort_values(by = "CT_values", ascending = True)
    return df_sorted


def find_missing_values_in_C():
    index_list = df[df["C (mg/L)"].isnull()].index.tolist()
    # Note: index 0 would not be a missing value

def drop_nan_last_rows(df):
    row_index = np.nan
    last_row_isnull = df[["C (mg/L)"]].iloc[-1:].isnull().values[0][0]
    if last_row_isnull == True:
        for i in range(1,len(df)+1):
            nan_bool = df[["C (mg/L)"]].iloc[-i:].isnull().all()[0]
            if nan_bool == True:
                row_index = -i
            else: 
                row_index = row_index
        df_drop = df.iloc[row_index:, :]
        df = pd.concat([df, df_drop]). drop_duplicates(keep = False)
    else:
        df = df
    return df, row_index

# Fill in missing values in C (mg/L)
def fill_in_missing_C(dff):
    
    original_shape = dff.shape
    
    # Drop the last row(s) if there are missing values in C (mg/L)
    dff, row_index = drop_nan_last_rows(dff)

    # Shape of df after dropping the missing value(s) at the end
    after_drop_shape = dff.shape

    index_list = np.where(dff["C (mg/L)"].isnull())[0].tolist()

    # Get the C (mg/L) on top of each missing value
    top_value_list = []
    j_top = 0
    top_index = []
    while (j_top < len(index_list)):
        i = index_list[j_top]
        while (dff[["C (mg/L)"]].isnull()["C (mg/L)"][i]):
            top = dff["C (mg/L)"][i-1] 
            i = i - 1
        top_index.append(i)
        top_value_list.append(top)
        j_top += 1

    # Get the C (mg/L) right below each missing value
    down_value_list = []
    j_down = 0
    down_index = []
    while (j_down < len(index_list)):
        i = index_list[j_down]
        while (dff[["C (mg/L)"]].isnull()["C (mg/L)"][i]):
            down = dff["C (mg/L)"][i+1] 
            i = i + 1
        down_index.append(i)
        down_value_list.append(down)
        j_down += 1

    # Get the time before and after each missing value
    top_time = []
    down_time = []
    missing_time_list = []
    for t, d, m in zip(top_index, down_index, index_list):
        top_time.append(dff ["Time (min)"][t])
        down_time.append(dff ["Time (min)"][d])
        missing_time_list.append(dff["Time (min)"][m])

    # Summary
    view_dict = {"index": index_list,
         "missing_time": missing_time_list,
         "top": top_value_list,
         "top_time_index": top_index,
         "top_time": top_time,
         "down": down_value_list,
         "down_time_index": down_index,
         "down_time": down_time}

    # Apply y = mx + b to get the missing C (mg/L)
    missing_C_list = []
    for i, missing_t in enumerate(missing_time_list):
        x1 = top_time[i]
        y1 = top_value_list[i]

        x2 = down_time[i]
        y2 = down_value_list[i]

        slope = (y2-y1)/(x2-x1)
        y = slope*(missing_t - x1) + y1
        missing_C_list.append(y)
    
    fill = pd.DataFrame(index =dff[["C (mg/L)"]].index[dff[["C (mg/L)"]].isnull().any(axis=1)], data= missing_C_list, columns=["C (mg/L)"])
    dff = dff.fillna(fill)
    return dff

# Given K values

In [3]:
# If k values are given, one data point should only have one k value (thus return index 0)
def Given_K(df):
    return df['Given_k'][0]

# Chlorine Decay

In [4]:
######## Main Function: Chlorine Decay #########################################################################################

# NOTE: 
# have_k_prime_already = False if k_prime is not given
# have_k_prime_already is number if k_prime is given

def chlorine_decay(df, have_k_prime_already, C_init, CT_to_use):
    
    # Fill in missing values if there is any in C (mg/L)
    if (df["C (mg/L)"].isnull().sum() == len(df)) | (df["C (mg/L)"].isnull().sum() == 0):
        pass
    else: 
        df = fill_in_missing_C(df)

    if have_k_prime_already == False:
        # Will use columns: Time (min), ln(C/C0), log 10 reduction + ln 10 reduction + k_ + k_lag + k_variation + CT_values
        df["ln(C/C0)"] = np.log(df["C (mg/L)"].astype('float') / df["C (mg/L)"][0])

    ############################ Calculate k prime for Approcah 1, 2, 3 ##########################################

    if have_k_prime_already != False :
        k_prime_3 = have_k_prime_already
        
    # Get C0
    # Note: Do NOT put only the initial conc in C (mg/L) column because the code will drop rows with missing conc
    if (df["CT_values"].isnull().sum() == len(df)) & (df["C (mg/L)"].isnull().sum() != len(df)):
        C_0 = df['C (mg/L)'][0]
        
    if (df["CT_values"].isnull().sum() == len(df)) & (df["C (mg/L)"].isnull().sum() == len(df)):
        C_0 = C_init

    chlorine = df['C (mg/L)']
    time = df['Time (min)']
    # Approach 1 (Do not use this approach because too much variation)
#     CT_values_1 = []; CT_values_1.append(0); k_prime_values_1 = []; delta_ts = []
#     sum_CT = 0; i = 0
#     while i+1 < len(time):
#         #find k value
#         delta_t = time[i+1]-time[i]
#         #check that times are correct: delta_ts.append(delta_t)
#         k_prime = abs(-ln(chlorine[i+1]/chlorine[i])/(delta_t))
#         k_prime_values_1.append(k_prime)
#         #find CT
#         sum_CT = sum_CT + (chlorine[i]/k_prime)*(1-np.exp(-k_prime*(delta_t)))
#         CT_values_1.append(sum_CT)
#         i = i+1

#     df["CT_values_1"] = CT_values_1
    df["CT_values_1"] = np.nan
    
    # Approach 2 (Only use this approach when k' is not provided)
    # We either assume the chlorine is constant or use the trapezoid calculation
    CT_values_2 = []; CT_values_2.append(0)
    sum_CT = 0; j = 0
    while j+1 < len(time):
        delta_t = time[j+1]-time[j]
        sum_CT = sum_CT  + (chlorine[j] + chlorine[j+1])/2*delta_t
        CT_values_2.append(sum_CT)
        j = j+1
    if have_k_prime_already == False :
        df["CT_values_2"] = CT_values_2
    else: 
        df["CT_values_2"] = np.nan

    # Approach 3 (Always use this approach when k' is provided)
    if have_k_prime_already != False :
        df["CT_values_3"] = (C_0/k_prime_3)*(1-np.exp(-k_prime_3* df['Time (min)']))
        CT_approach_num = 3
        CT = "CT_values_3"
    else: 
        df["CT_values_3"] = np.nan
        CT_approach_num = 2
        CT = "CT_values_2"

    # Convert to ln 10 reduction
    convert_to_ln(df)

    # Sort by Time (min) column
    df = sort_row_according_to_Time(df)
       
    coef_list = []; R2_list = []

    coef = get_coef(df, CT_approach_num, CT, CT_to_use)

    coef_list.append(coef)
    
    X = np.array(df[CT])
    Y = np.array(df['ln 10 reduction'])
    
    # Modified the conditions on 3/4/2023
    if (((CT_approach_num == 2) & (coef !=0)) | ((CT_approach_num == 3) & (coef !=0))):   
        df_xy = pd.DataFrame({"x_val": X[0:CT_to_use], "y_val": Y[0:CT_to_use]})
        smOLS_reault = sm.OLS(endog= df_xy[['y_val']], exog= df_xy[['x_val']].assign(intercept=0)).fit()

        coef_ols = smOLS_reault.params[0]
        error = smOLS_reault.bse[0]
            
    # Empty df to prevent the old values appear on the next run
    empty_df(df)
    
    coef = np.abs(coef)
    coef_ols = np.abs(coef_ols)
    return coef, coef_ols, error   


# Constant Chlorine

In [5]:
### < Code for Constant Chlorine > ###

# NOTE: concentration = const_C
def regression_for_CT(df, CT_to_use):
    coef_list = []
    if (df["ln 10 reduction"].isnull().sum() != 0):
        data = df.dropna(subset = ['ln 10 reduction'])
    else: 
        data = df.copy()
    
    x = np.array(data["CT_values"][0:CT_to_use])
    y = np.array(data['ln 10 reduction'][0:CT_to_use])
    
    df_xy = pd.DataFrame({"x_val": x, "y_val": y})
    smOLS_reault = sm.OLS(endog= df_xy[['y_val']], exog= df_xy[['x_val']].assign(intercept=0)).fit()

    k_value_OLS = smOLS_reault.params[0]
    std_erro_OLS = smOLS_reault.bse[0]
    
    A = np.vstack([x, np.ones(len(x))]).T # Add one to x and drop the ones later
    k_value = float(np.linalg.lstsq(A[:, :-1], y)[0]) # use [0] to just grab the k value coefs
    
    df.iloc[0:CT_to_use, :]
    
    return k_value, k_value_OLS, std_erro_OLS # (2/19/2023) Also return k_value_OLS, std_erro_OLS here 

def constant_chlorine(df, concentration, CT_to_use):
    df["CT_values"] = df["Time (min)"] * concentration
    df = sort_row_according_to_Time(df) # Sort Time column
    convert_to_ln(df)
    
    # Visualize the linear regression
    k_value, k_value_OLS, std_erro_OLS = regression_for_CT(df, CT_to_use)
    k_value = np.abs(k_value)
    k_value_OLS = np.abs(k_value_OLS)

    # Empty df to prevent the old values appear on the next run
    empty_df(df)
    
    return k_value, k_value_OLS, std_erro_OLS

# Given CT

In [6]:
def Given_CT(df, CT_to_use):    
    convert_to_ln(df)

    # Sort by CT columns
    df = df.sort_values(by = "CT_values", ascending = True)
    # display(df) # testing
   
    # Visualize the linear regression
    k_value, k_value_OLS, std_erro_OLS = regression_for_CT(df, CT_to_use)
    k_value = np.abs(k_value)
    k_value_OLS = np.abs(k_value_OLS)

    # Empty df to prevent the old values appear on the next run
    empty_df(df)
    
    return k_value, k_value_OLS, std_erro_OLS

# Applications Start Here

In [7]:
R1path = '/Users/mirac/Desktop/Chlorine_Project/August 18 Chlorine Review Paper/Chlorine_Review_Revisions/1-Revisions_Inactivation_Rate_Data_R1.xlsx'
R2path = '/Users/mirac/Desktop/Chlorine_Project/August 18 Chlorine Review Paper/Chlorine_Review_Revisions/2-Revisions_Inactivation_Rate_Data_R2.xlsx'

#insert sheet name here
sheet_name = "Paper_32"
summ = results(R2path, sheet_name)
greaterSmallerCols = [col for col in summ.columns if "_than" in col]
summ[greaterSmallerCols] = summ[greaterSmallerCols].fillna(value=0)
display(summ) 

,Sample_ID,Method,Coef_np,Coef_OLS,Standard_Error,Data_Points_Used,Diff,larger_than,smaller_than
0,"""32-1""",3,179.923163,179.923163,8.917193,22,0,0.0,0.0
1,"""32-2""",3,1.493117,1.493117,0.116947,21,0,0.0,0.0


# Combine Every Paper

In [8]:
initalValPath = '/Users/mirac/Desktop/Chlorine_Project/August 18 Chlorine Review Paper/Chlorine_Review_Revisions/Virus_Data - Final_Experimental_Data.csv'
numArray = np.arange(1, 82, 1)
# finalTable = pd.DataFrame(columns = ['Sample_ID', 'Method', 'Coef_np', 'Coef_OLS', 'Standard_Error', 'Diff', 'Data_Points_Used'])
finalTable_R1 = pd.DataFrame()

path_to_use = R1path # Change path here

for num in numArray:
    sheetName = "Paper_" + str(num)
    summ = results(path_to_use, sheetName) 
    
    # Rename greater_than to largr_than
    if "greater_than" in summ.columns:
        summ.rename(columns = {'greater_than': 'larger_than'}, inplace=True)

    greaterSmallerCols = [col for col in summ.columns if "_than" in col]
    summ[greaterSmallerCols] = summ[greaterSmallerCols].fillna(value=0)
    finalTable_R1 = pd.concat([finalTable_R1, summ], axis = 0)
    # print("Paper ", num, ": Done") # testing

In [9]:
finalTable_R2 = pd.DataFrame()
path_to_use = R2path # Change path here

for num in numArray:
    sheetName = "Paper_" + str(num)
    summ = results(path_to_use, sheetName) 
    
    # Rename greater_than to largr_than
    if "greater_than" in summ.columns:
        summ.rename(columns = {'greater_than': 'larger_than'}, inplace=True)

    greaterSmallerCols = [col for col in summ.columns if "_than" in col]
    summ[greaterSmallerCols] = summ[greaterSmallerCols].fillna(value=0)
    finalTable_R2 = pd.concat([finalTable_R2, summ], axis = 0)
    # print("Paper ", num, ": Done") # testing

In [10]:
def removeQuotes(x):
    return x.replace('"', "")

finalTable_R1['Sample_ID'] = finalTable_R1['Sample_ID'].apply(removeQuotes)
finalTable_R2['Sample_ID'] = finalTable_R2['Sample_ID'].apply(removeQuotes)

# Combine Mira and Kaming Tables

In [11]:
finalTable_R1_2 = finalTable_R1.copy()
finalTable_R1_2 = finalTable_R1_2[['Sample_ID', 'Method', 'Coef_np', 'Coef_OLS', 'Standard_Error',
       'Data_Points_Used', 'Diff', 'Data_Points', 'smaller_than']]
keep_same = {'Sample_ID'}
finalTable_R1_2.columns = ['{}{}'.format(c, '' if c in keep_same else '_R1')
               for c in finalTable_R1_2.columns]

finalTable_R1_2.columns

#finalTableMira_2.to_csv('finalTable_R1_2.csv', index=False)


Index(['Sample_ID', 'Method_R1', 'Coef_np_R1', 'Coef_OLS_R1',
       'Standard_Error_R1', 'Data_Points_Used_R1', 'Diff_R1', 'Data_Points_R1',
       'smaller_than_R1'],
      dtype='object')

In [12]:
finalTable_R2_2 = finalTable_R2.copy()
finalTable_R2_2 = finalTable_R2_2[['Sample_ID', 'Method', 'Coef_np', 'Coef_OLS', 'Standard_Error',
       'Data_Points_Used', 'Diff', 'smaller_than', 'larger_than']]
finalTable_R2_2.columns = ['{}{}'.format(c, '' if c in keep_same else '_R2')
               for c in finalTable_R2_2.columns]


In [13]:
dataset_both_reviewers = finalTable_R1_2.merge(finalTable_R2_2, on = "Sample_ID", how = 'left')

dataset_both_reviewers.to_csv('4-kinetics_data.csv', index=False)


# Compare with initial values

In [14]:
initial = pd.read_csv(initalValPath)[['kobs_mira (removed > and <)', 'kobs_kaming (removed > and <)', 'Sample ID']]
initial.dropna(subset= ['kobs_mira (removed > and <)', 'kobs_kaming (removed > and <)'], inplace=True)
initial.columns = ['init_mira', 'init_kaming', 'Sample_ID']
print(initial.shape)

# Data points used for modeling
data_used_model = pd.read_csv(initalValPath)
data_used_modeling = data_used_model[data_used_model['data_for_modeling']==1]['Sample ID'].values

(636, 3)


In [15]:
merged_init = finalTable_R1.merge(initial, on = "Sample_ID", how = 'left')

diffCheck = merged_init['Diff'] != 0
if (diffCheck.any()):
    raise ValueError("Column 'Diff' is non-zero")

if (path_to_use == R1path):
    init_col = 'init_kaming'
elif (path_to_use == R2path):
    init_col = 'init_mira'
else:
    raise ValueError("ERROR: Unknow file path!")
print(f"Path used: {path_to_use}")

merged_init['CompareInit'] = np.abs(np.abs(merged_init['Coef_np']) - np.abs(merged_init[init_col]))
merged_init_compare = merged_init[merged_init['CompareInit'] > (10**-6)] # Set threshold at 10^(-6)

merged_init_compare = merged_init_compare[merged_init_compare['larger_than'] == 0]
merged_init_compare = merged_init_compare[merged_init_compare["Sample_ID"].isin(data_used_modeling)]
print(f"There are {merged_init_compare.shape[0]} rows with k value differences > 10^-6.")

merged_init_compare

Path used: /Users/mirac/Desktop/Chlorine_Project/August 18 Chlorine Review Paper/Chlorine_Review_Revisions/2-Revisions_Inactivation_Rate_Data_R2.xlsx
There are 0 rows with k value differences > 10^-6.


,Sample_ID,Method,Coef_np,Coef_OLS,Standard_Error,Data_Points_Used,Diff,larger_than,Data_Points,smaller_than,...,Unnamed: 21,Unnamed: 26,Unnamed: 25,Unnamed: 27,Unnamed: 23,Unnamed: 22,Unnamed: 28,init_mira,init_kaming,CompareInit


In [16]:
# Search for specific paper
paper = 21
merged_init_compare[merged_init_compare['Sample_ID'].str.startswith(str(paper)+ "-")]

,Sample_ID,Method,Coef_np,Coef_OLS,Standard_Error,Data_Points_Used,Diff,larger_than,Data_Points,smaller_than,...,Unnamed: 21,Unnamed: 26,Unnamed: 25,Unnamed: 27,Unnamed: 23,Unnamed: 22,Unnamed: 28,init_mira,init_kaming,CompareInit


In [17]:
# Search for specificpaper
paper = 21
merged_selection = merged_init_compare[merged_init_compare['Sample_ID'].str.startswith(str(paper)+ "-")]
print(merged_selection[['Sample_ID','Coef_np','init_kaming','CompareInit']])

Empty DataFrame
Columns: [Sample_ID, Coef_np, init_kaming, CompareInit]
Index: []


### Sort the k value differences in a descending order

In [18]:
pd.set_option('display.max_rows', 1000)
diff_df = merged_init_compare.sort_values(by = 'CompareInit', ascending = False)
print(diff_df.shape)

diff_df[diff_df['CompareInit'] > 10**(-6)].shape
diff_df

# Paper 19 not included in modeling

(0, 24)


,Sample_ID,Method,Coef_np,Coef_OLS,Standard_Error,Data_Points_Used,Diff,larger_than,Data_Points,smaller_than,...,Unnamed: 21,Unnamed: 26,Unnamed: 25,Unnamed: 27,Unnamed: 23,Unnamed: 22,Unnamed: 28,init_mira,init_kaming,CompareInit


In [19]:
### Tailing Analysis

In [20]:
finalTable_R1.columns

Index(['Sample_ID', 'Method', 'Coef_np', 'Coef_OLS', 'Standard_Error',
       'Data_Points_Used', 'Diff', 'larger_than', 'Data_Points',
       'smaller_than', 'Unnamed: 20', 'Unnamed: 19', 'Unnamed: 18',
       'Unnamed: 24', 'Unnamed: 21', 'Unnamed: 26', 'Unnamed: 25',
       'Unnamed: 27', 'Unnamed: 23', 'Unnamed: 22', 'Unnamed: 28'],
      dtype='object')